# 1. The Multiple Roles of Anticipation in Developmental Robotics

Replicating the experiments in https://www.aaai.org/Papers/Symposia/Fall/2005/FS-05-05/FS05-05-002.pdf

First, we import `conx` and create a 6-10-2+2 feed-forward network:

In [2]:
from calysto.ai.conx import Network
import random

In [3]:
class XorNetwork(Network):
    def __init__(self):
        super().__init__()
        self.totalCorrect = 0
        self.totalPossible = 0
        self.totalTSSError = 0
        
    def correct(self, layername):
        c = 0
        for i in range(len(self[layername].activation)):
            if abs(self[layername].activation[i] - self[layername].target[i]) < self.tolerance:
                c += 1
        return c

    def reportPattern(self):
        if (self["input"].activation[0] == 1 and
            self["input"].activation[1] == 1):
            c = self.correct("output")
            self.totalCorrect += c
            self.totalPossible += 2
            self.totalTSSError += self["output"].TSSError()
            
    def reportEpoch(self, *args):
        print("Epoch:", self.epoch, end=" ")
        print("Percent Correct:", self.totalCorrect/float(self.totalPossible), end=" ")
        print("TSS Error:", self.totalTSSError)
        self.totalCorrect = 0
        self.totalPossible = 0
        self.totalTSSError = 0

In [4]:
net = XorNetwork()

Conx using seed: 1437735633.2086067


In [5]:
net.addLayer("input", 6)
net.addLayer("hidden", 10)
net.addLayer("output", 2)
net.addLayer("hints", 2)

net.connect("input", "hidden")
net.connect("hidden", "output")
net.connect("hidden", "hints")

In [6]:
net.setTolerance(.4) # outputs must be within this of targets to consider correct
net.setMomentum(0.0)
net.setEpsilon(.6)
net.resetCount = 1
net.resetEpoch = 1000
net.setStopPercent(1.1) # don't stop

We write a couple of helper functions to generate the inputs and targets:

In [7]:
def bin2list(n, width):
    return [int(b) for b in (("%0" + str(width) + "d") % int(bin(n)[2:]))]

Test `bin2list`:

In [8]:
bin2list(1, 6)

[0, 0, 0, 0, 0, 1]

In [9]:
def xor(a, b):
    return a ^ b

Test `xor`:

In [10]:
for group in range(4):
    list_group = bin2list(group, 2)
    print(list_group, ":", xor(*list_group))

[0, 0] : 0
[0, 1] : 1
[1, 0] : 1
[1, 1] : 0


In [11]:
inputs = []
targets = []
for group in range(4):
    list_group = bin2list(group, 2)
    for i in range(2 ** 4):
        list_i = bin2list(i, 4)
        inputs.append(list_group +  list_i)
        targets.append([xor(*list_i[0:2]), xor(*list_i[2:4])] + [0.5, 0.5])


In [12]:
len(inputs), len(targets)

(64, 64)

In [13]:
net.mapTarget("output", 0)
net.mapTarget("hints", 2)

Now we are ready to set the inputs and targets and train:

In [14]:
net.setInputs(inputs)

In [15]:
net.setTargets(targets)

## 1.1 Standard XOR Problem

In [16]:
net.initialize()

Initializing 'Backprop Network' weights... 

In [17]:
%%time
net.train()

Epoch: 25 Percent Correct: 0.085 TSS Error: 210.387959588
Epoch: 50 Percent Correct: 0.0825 TSS Error: 206.114479848
Epoch: 75 Percent Correct: 0.42375 TSS Error: 172.805874726
Epoch: 100 Percent Correct: 0.48125 TSS Error: 161.506952252
Epoch: 125 Percent Correct: 0.9725 TSS Error: 20.4823148419
Epoch: 150 Percent Correct: 1.0 TSS Error: 1.61885719386
Epoch: 175 Percent Correct: 1.0 TSS Error: 0.635201165256
Epoch: 200 Percent Correct: 1.0 TSS Error: 0.347357788025
Epoch: 225 Percent Correct: 1.0 TSS Error: 0.224530690366
Epoch: 250 Percent Correct: 1.0 TSS Error: 0.157763387438
Epoch: 275 Percent Correct: 1.0 TSS Error: 0.116869609826
Epoch: 300 Percent Correct: 1.0 TSS Error: 0.0906929986352
Epoch: 325 Percent Correct: 1.0 TSS Error: 0.0724010081715
Epoch: 350 Percent Correct: 1.0 TSS Error: 0.0594036871919
Epoch: 375 Percent Correct: 1.0 TSS Error: 0.0494409150958
Epoch: 400 Percent Correct: 1.0 TSS Error: 0.0419186327269
Epoch: 425 Percent Correct: 1.0 TSS Error: 0.0360186050152
E

Learns the patterns very quickly, usually in less that 100 epochs.

In [17]:
net.interactive = 1
net.sweep()
net.interactive = 0

-----------------------------------Pattern # 49
Display network 'Backprop Network':
Layer 'hints': (Kind: Output, Size: 2, Active: 1, Frozen: 0)
Target    :  0.50 0.50 
Activation:  0.50 0.50 
Layer 'output': (Kind: Output, Size: 2, Active: 1, Frozen: 0)
Target    :  0.00 1.00 
Activation:  0.00 1.00 
Layer 'hidden': (Kind: Hidden, Size: 10, Active: 1, Frozen: 0)
Activation:  1.00 0.06 0.23 0.03 0.05 0.17 0.96 0.16 0.10 0.00 
Layer 'input': (Kind: Input, Size: 6, Active: 1, Frozen: 0)
Activation:  1.00 1.00 0.00 0.00 0.00 1.00 
<q>uit, <g>o, or press Enter: 
-----------------------------------Pattern # 20
Display network 'Backprop Network':
Layer 'hints': (Kind: Output, Size: 2, Active: 1, Frozen: 0)
Target    :  0.50 0.50 
Activation:  0.50 0.50 
Layer 'output': (Kind: Output, Size: 2, Active: 1, Frozen: 0)
Target    :  0.00 0.00 
Activation:  0.00 0.00 
Layer 'hidden': (Kind: Hidden, Size: 10, Active: 1, Frozen: 0)
Activation:  0.13 0.04 0.05 0.90 0.00 0.06 0.00 0.04 0.02 0.89 
Layer

## 1.2 XOR in Noisy Environment

Now, we will make the task a bit harder by corrupting some patterns so that they cannot be learned:

In [27]:
def postPropagate(*args, **kwargs):
    input_pattern = kwargs["input"]
    if input_pattern[:2] in [[0, 0], [0, 1], [1, 0]]:
        for i in range(2):
            kwargs["output"][i] = random.random() #random.randint(0, 1)
    target_pattern = kwargs["hints"]
    for i in range(2):
        error = net["output"][i].activation - kwargs["output"][i]
        target_pattern[i] = abs(error) # max(min((error + 1.0)/2.0, 1.0), 0.0)
    return kwargs

net.postPropagate = postPropagate

If an input starts with [0,0], [0,1] or [1,0] then we randomly change the targets.

In [28]:
net.initialize()

Initializing 'Backprop Network' weights... 

In [29]:
%%time
net.train()

Epoch: 25 Percent Correct: 0.48165760869565216 TSS Error: 207.246010391
Epoch: 50 Percent Correct: 0.015 TSS Error: 207.664382012
Epoch: 75 Percent Correct: 0.0225 TSS Error: 207.731268183
Epoch: 100 Percent Correct: 0.01 TSS Error: 207.819999006
Epoch: 125 Percent Correct: 0.025 TSS Error: 206.749954314
Epoch: 150 Percent Correct: 0.0175 TSS Error: 206.341989881
Epoch: 175 Percent Correct: 0.0175 TSS Error: 207.682095538
Epoch: 200 Percent Correct: 0.0175 TSS Error: 205.65654072
Epoch: 225 Percent Correct: 0.02 TSS Error: 206.406838443
Epoch: 250 Percent Correct: 0.005 TSS Error: 207.952140679
Epoch: 275 Percent Correct: 0.01 TSS Error: 206.11341432
Epoch: 300 Percent Correct: 0.0175 TSS Error: 204.903412577
Epoch: 325 Percent Correct: 0.015 TSS Error: 207.163058142
Epoch: 350 Percent Correct: 0.01375 TSS Error: 205.657379616
Epoch: 375 Percent Correct: 0.02625 TSS Error: 207.758648002
Epoch: 400 Percent Correct: 0.01375 TSS Error: 207.236668269
Epoch: 425 Percent Correct: 0.0125 TSS 

Standard backprop cannot learn the pattern in over 3k+ epochs.

In [ ]:
net.interactive = 1
net.sweep()
net.interactive = 0

-----------------------------------Pattern # 44
Display network 'Backprop Network':
Layer 'hints': (Kind: Output, Size: 2, Active: 1, Frozen: 0)
Target    :  0.55 0.37 
Activation:  0.51 0.48 
Layer 'output': (Kind: Output, Size: 2, Active: 1, Frozen: 0)
Target    :  0.00 0.00 
Activation:  0.55 0.37 
Layer 'hidden': (Kind: Hidden, Size: 20, Active: 1, Frozen: 0)
Activation:  0.02 0.03 0.02 0.02 0.02 0.02 0.02 0.02 0.23 0.01 0.02 0.02 0.02 0.02 0.02 0.04 0.02 0.02 0.02 0.02 
Layer 'input': (Kind: Input, Size: 6, Active: 1, Frozen: 0)
Activation:  1.00 0.00 1.00 0.00 0.00 0.00 
<q>uit, <g>o, or press Enter: 
-----------------------------------Pattern # 57
Display network 'Backprop Network':
Layer 'hints': (Kind: Output, Size: 2, Active: 1, Frozen: 0)
Target    :  0.48 0.28 
Activation:  0.51 0.45 
Layer 'output': (Kind: Output, Size: 2, Active: 1, Frozen: 0)
Target    :  1.00 0.00 
Activation:  0.52 0.28 
Layer 'hidden': (Kind: Hidden, Size: 20, Active: 1, Frozen: 0)
Activation:  0.01 0

<div class="alert alert-danger">

This is a test.

</div>

<div class="alert alert-danger">

This is a test.

</div>